In [8]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


def loadNclean():
    data = pd.read_csv('training.csv')
    data['tweet'] = data['tweet'].apply(lambda x: x.lower())
    data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    return data

max_words = 500
data = loadNclean()
tok = Tokenizer(nb_words=max_words, split=' ')
tok.fit_on_texts(data['tweet'].values)
X = tok.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = 32
lstm_out = 10
def buildModel(): 
    model = Sequential()
    model.add(Embedding(max_words,embed_dim,input_length=X.shape[1]))
    model.add(LSTM(lstm_out))
    model.add(Dense(2))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
model = buildModel()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 92, 32)            16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                1720      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
Total params: 17,742
Trainable params: 17,742
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
Y = np.array(data)[:,1:3]
Y = np.asarray(Y).astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape,Y_train.shape)

(15130, 92) (15130, 2)


In [19]:
model.fit(X_train,Y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
473/473 [==============================] - 9s 19ms/step - loss: 1040.8303
Epoch 2/10
473/473 [==============================] - 9s 18ms/step - loss: 754.5096
Epoch 3/10
473/473 [==============================] - 9s 19ms/step - loss: 574.3218
Epoch 4/10
473/473 [==============================] - 9s 19ms/step - loss: 431.1081
Epoch 5/10
473/473 [==============================] - 9s 19ms/step - loss: 315.6160
Epoch 6/10
473/473 [==============================] - 9s 18ms/step - loss: 223.6107
Epoch 7/10
473/473 [==============================] - 9s 19ms/step - loss: 151.9853
Epoch 8/10
473/473 [==============================] - 9s 19ms/step - loss: 98.0467
Epoch 9/10
473/473 [==============================] - 9s 19ms/step - loss: 59.2388
Epoch 10/10
473/473 [==============================] - 9s 19ms/step - loss: 33.0004
